# Influence des politiques agricoles sur la souverainete alimentaire  
## Analyse par pays  
### Produits : Volaille, Ble, Lait

Ce notebook charge et prepare les donnees de **production** et **importations** agricoles
afin de calculer un indicateur de **souverainete alimentaire** (taux d'autosuffisance).

Sources prevues :
- FAOSTAT - Production
- FAOSTAT - Importations

Periode : aussi longue que possible (idealement >= 1970)


In [2]:

import pandas as pd
import numpy as np
from pathlib import Path

BASE = Path.cwd()

## Parametrage : produits

Les codes produits correspondent aux *Item Codes* FAOSTAT.


In [3]:
# Codes FAOSTAT des produits
PRODUCTS = {
    'ble': 111,        # Wheat
    'volaille': 21121, # Meat of chickens, fresh or chilled (CPC)
    'lait': 2211       # Raw milk of cattle (CPC)
}



## Chargement des données de production

Fichier attendu :
- `FAOSTAT_production.csv`
- Source : FAOSTAT – *Crops and Livestock Products* https://www.fao.org/faostat/en/#data/QCL


In [6]:
PRODUCTION_FILE = BASE / 'data/FAOSTAT_production.csv'

prod_raw = pd.read_csv(PRODUCTION_FILE)

df_prod = (
    prod_raw
    .query('`Item Code (CPC)` in @PRODUCTS.values()')
    .rename(columns={
        'Area': 'country',
        'Item': 'product',
        'Year': 'year',
        'Value': 'production_tonnes'
    })
    [['country', 'product', 'year', 'production_tonnes']]
)

df_prod.head()


,country,product,year,production_tonnes
0,Norway,"Meat of chickens, fresh or chilled",1961,2477.0
1,Norway,"Meat of chickens, fresh or chilled",1962,2648.0
2,Norway,"Meat of chickens, fresh or chilled",1963,2465.0
3,Norway,"Meat of chickens, fresh or chilled",1964,2535.0
4,Norway,"Meat of chickens, fresh or chilled",1965,3222.0



## Chargement des données d’importations

Fichier attendu :
- `FAOSTAT_imports.csv`
- Source : FAOSTAT – *Trade*
- Élément : Import Quantity


In [7]:
IMPORTS_FILE = BASE / 'data/FAOSTAT_import.csv'

imp_raw = pd.read_csv(IMPORTS_FILE)

imports = (
    imp_raw
    .loc[lambda x: x['Item Code (CPC)'].isin(PRODUCTS.values()) & (x['Element'] == 'Import quantity')]
    .rename(columns={
        'Area': 'country',
        'Item': 'product',
        'Year': 'year',
        'Value': 'imports_tonnes'
    })
    [['country', 'product', 'year', 'imports_tonnes']]
)

imports.head()


,country,product,year,imports_tonnes
0,Norway,"Meat of chickens, fresh or chilled",1961,39.0
2,Norway,"Meat of chickens, fresh or chilled",1962,45.0
4,Norway,"Meat of chickens, fresh or chilled",1963,59.0
6,Norway,"Meat of chickens, fresh or chilled",1964,124.0
8,Norway,"Meat of chickens, fresh or chilled",1965,120.0



## Fusion production / importations


In [8]:
df = (
    df_prod
    .merge(
        imports,
        on=['country', 'product', 'year'],
        how='inner'
    )
)

df.head()


,country,product,year,production_tonnes,imports_tonnes
0,Norway,"Meat of chickens, fresh or chilled",1961,2477.0,39.0
1,Norway,"Meat of chickens, fresh or chilled",1962,2648.0,45.0
2,Norway,"Meat of chickens, fresh or chilled",1963,2465.0,59.0
3,Norway,"Meat of chickens, fresh or chilled",1964,2535.0,124.0
4,Norway,"Meat of chickens, fresh or chilled",1965,3222.0,120.0



## Calcul du taux d’autosuffisance alimentaire


In [9]:

df["self_sufficiency_rate"] = (
    df["production_tonnes"] /
    (df["production_tonnes"] + df["imports_tonnes"])
)

df.describe()


,year,production_tonnes,imports_tonnes,self_sufficiency_rate
count,576.00000,5.760000e+02,5.760000e+02,576.000000
mean,1992.50000,2.703097e+07,2.822915e+06,0.793925
std,18.48901,4.919649e+07,7.019204e+06,0.246394
min,1961.00000,2.465000e+03,0.000000e+00,0.010632
25%,1976.75000,1.059358e+05,1.575975e+04,0.636169
50%,1992.50000,1.666500e+06,2.209935e+05,0.907580
75%,2008.25000,8.175591e+06,6.832338e+05,0.993769
max,2024.00000,1.551803e+08,3.999379e+07,1.000000



## Nettoyage et contrôles


In [10]:
# Garder uniquement les valeurs plausibles
df = df[df['self_sufficiency_rate'].between(0, 1)]

# Verifier la couverture temporelle
df.groupby(['country', 'product'])['year'].agg(['min', 'max'])


min   max
country             product                                       
European Union (27) Meat of chickens, fresh or chilled  1961  2024
                    Raw milk of cattle                  1961  2024
                    Wheat                               1961  2024
Norway              Meat of chickens, fresh or chilled  1961  2024
                    Raw milk of cattle                  1961  2024
                    Wheat                               1961  2024
Switzerland         Meat of chickens, fresh or chilled  1961  2024
                    Raw milk of cattle                  1961  2024
                    Wheat                               1961  2024


## Sauvegarde du dataset préparé


In [11]:

OUTPUT_FILE = "data/agri_self_sufficiency_prepared.csv"
df.to_csv(OUTPUT_FILE, index=False)

OUTPUT_FILE


'data/agri_self_sufficiency_prepared.csv'

### Resultat

Le dataset final contient :
- country
- product
- year
- production_tonnes
- imports_tonnes
- self_sufficiency_rate

Il est pret pour :
- visualisations temporelles (small multiples),
- comparaisons pays par pays,
- analyses exploratoires.
